In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer

import numpy as np
import tensorflow as tf
import tflearn
import random
import json
from google.colab import files
uploaded =files.upload()
with open("intents.json") as file:
  data=json.load(file)
print(data)

Instructions for updating:
non-resource variables are not supported in the long term


Saving intents.json to intents.json
{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolat

In [2]:
pip install tflearn

     |████████████████████████████████| 107 kB 4.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=614161d49968b290fa810a3050140c6f3eeda381425ee4df8b6b4a8b0fbf3386
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
i

In [27]:
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds=nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent['tag'])
    if intent["tag"] not in labels:
      labels.append(intent["tag"])
print(words)
words1=[]
stemmer=LancasterStemmer()
for w in words:
  words1.append(stemmer.stem(w.lower()))
#[stemmer.stem(w.lower()) for w in words if w != "?"]
words =sorted(list(set(words1)))
labels=sorted(labels)
training =[]
output=[]
out_empty=[0 for _ in range(len(labels))]
 
for x,doc in enumerate(docs_x):
  bag=[]
  wrds=[stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)
  output_row=out_empty[:]
  output_row[labels.index(docs_y[x])] =1
  training.append(bag)
  output.append(output_row)
training=np.array(training)
output=np.array(output)

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation']


In [41]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net=tflearn.input_data(shape=[None,len(training[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(output[0]),activation='softmax')
net=tflearn.regression(net)

model=tflearn.DNN(net)

model.fit(training,output,n_epoch=1000,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.01159 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.01159 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.01171 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.01171 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [42]:
def bag_of_words(s,words):
  bag=[0 for _ in range(len(words))]

  s_words=nltk.word_tokenize(s)
  s_words=[stemmer.stem(word.lower()) for word in s_words]
  for se in s_words:
    for i,w in enumerate(words):
      if w==se:
        bag[i]=1
  return np.array(bag)

In [43]:
def chat():
  print("Start talking with the bot (type quit to stop")
  while True:
    inp=input("You :")
    if inp.lower()=='quit':
      break
    results=model.predict([bag_of_words(inp,words)])[0]
    results_index=np.argmax(results)
    tag=labels[results_index]

    if results[results_index] >0.7:
      for tg in data['intents']:
        if tg['tag']==tag:
          responses=tg['responses']
      print(random.choice(responses))
    else:
      print("I didn't get that, try again")
    

In [44]:
chat()

Start talking with the bot (type quit to stop
You :hello
Hello!
You :Hi
Good to see you again!
You :makes no sense
Good to see you again!
You :It makes no sense
Good to see you again!
You :sfseef
Hello!
You :seffsefsef
Hello!
You :Your age
18 years young!
You :Your occupation
I didn't get that, try again
You :your mental health
I didn't get that, try again
You :sef
Hi there, how can I help?
You :whatsapp
Good to see you again!
You :konichwa
Good to see you again!
You :sayonara
Hi there, how can I help?
You :goodbye
Sad to see you go :(
You :quit


In [22]:
import numpy as np
import tensorflow

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True